# Main App

In [55]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
#|export
import pandas as pd
from flask import Flask, redirect, request
from flask import render_template
import json

In [57]:
people = [
    {
        "firstname": "John",
        "lastname": "Doe",
        "phone": "555-1234",
        "email": "john.doe@example.com"
    },
    {
        "firstname": "Jane",
        "lastname": "Smith",
        "phone": "555-5678",
        "email": "jane.smith@example.com"
    },
    {
        "firstname": "Alice",
        "lastname": "Johnson",
        "phone": "555-8765",
        "email": "alice.johnson@example.com"
    },
    {
        "firstname": "Bob",
        "lastname": "Williams",
        "phone": "555-4321",
        "email": "bob.williams@example.com"
    },
    {
        "firstname": "Charlie",
        "lastname": "Brown",
        "phone": "555-9876",
        "email": "charlie.brown@example.com"
    },
    {
        "firstname": "Diana",
        "lastname": "Miller",
        "phone": "555-6543",
        "email": "diana.miller@example.com"
    },
    {
        "firstname": "Edward",
        "lastname": "Davis",
        "phone": "555-3456",
        "email": "edward.davis@example.com"
    },
    {
        "firstname": "Fiona",
        "lastname": "Garcia",
        "phone": "555-6789",
        "email": "fiona.garcia@example.com"
    },
    {
        "firstname": "George",
        "lastname": "Martinez",
        "phone": "555-7890",
        "email": "george.martinez@example.com"
    },
    {
        "firstname": "Hannah",
        "lastname": "Rodriguez",
        "phone": "555-8901",
        "email": "hannah.rodriguez@example.com"
    }
]

# Writing the JSON data to a file
file_path = 'contactdb.json'
with open(file_path, 'w') as json_file:
    json.dump(people, json_file, indent=4)

file_path

'contactdb.json'

In [58]:
db = pd.read_json('contactdb.json')
db.to_dict('records')

[{'firstname': 'John',
  'lastname': 'Doe',
  'phone': '555-1234',
  'email': 'john.doe@example.com'},
 {'firstname': 'Jane',
  'lastname': 'Smith',
  'phone': '555-5678',
  'email': 'jane.smith@example.com'},
 {'firstname': 'Alice',
  'lastname': 'Johnson',
  'phone': '555-8765',
  'email': 'alice.johnson@example.com'},
 {'firstname': 'Bob',
  'lastname': 'Williams',
  'phone': '555-4321',
  'email': 'bob.williams@example.com'},
 {'firstname': 'Charlie',
  'lastname': 'Brown',
  'phone': '555-9876',
  'email': 'charlie.brown@example.com'},
 {'firstname': 'Diana',
  'lastname': 'Miller',
  'phone': '555-6543',
  'email': 'diana.miller@example.com'},
 {'firstname': 'Edward',
  'lastname': 'Davis',
  'phone': '555-3456',
  'email': 'edward.davis@example.com'},
 {'firstname': 'Fiona',
  'lastname': 'Garcia',
  'phone': '555-6789',
  'email': 'fiona.garcia@example.com'},
 {'firstname': 'George',
  'lastname': 'Martinez',
  'phone': '555-7890',
  'email': 'george.martinez@example.com'},
 {'

In [79]:
q = '1234'
# db[db['firstname'].str.lower().str.contains(q)]
predicate_firstname = db['firstname'].str.lower().str.contains(q.lower())
predicate_lastname = db['lastname'].str.lower().str.contains(q.lower())
predicate_phone = db['phone'].str.lower().str.contains(q.lower())
predicate_email= db['email'].str.lower().str.contains(q.lower())
db[predicate_firstname|predicate_lastname|predicate_phone|predicate_email]

,firstname,lastname,phone,email
0,John,Doe,555-1234,john.doe@example.com


In [80]:
#|export
class Contact(object):
    def __init__(self) -> None:
        self.db = pd.read_json('contactdb.json')

    def search(self, q):
        predicate_firstname = self.db['firstname'].str.lower().str.contains(q.lower())
        predicate_lastname = self.db['lastname'].str.lower().str.contains(q.lower())
        predicate_phone = self.db['phone'].str.lower().str.contains(q.lower())
        predicate_email= self.db['email'].str.lower().str.contains(q.lower())
        filter_df = self.db[predicate_firstname|predicate_lastname|predicate_phone|predicate_email]
        return filter_df.to_dict('records')

    def all(self):
        return self.db.to_dict('records')

In [81]:
#|export

app = Flask(__name__)

In [82]:
#|export

@app.get("/")
def index():
    return redirect("/contacts")

In [83]:
#|export

@app.get("/contacts")
def contacts():
    search = request.args.get("q")
    contact_set = None
    if search is not None: contact_set = Contact().search(search)
    else: contact_set = Contact().all()
    # print(contact_set)
    return render_template("index.html", contact_set=contact_set)

In [84]:
#|eval:false
from nbdev.export import nb_export
nb_export("01_main.ipynb", lib_path=".", name='main')